In [7]:
import numpy as np
import pandas as pd
import itertools
import random
import time
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim


from sklearn import datasets
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [2]:
def display_chosen_labelled_datas_PCA(X_train,idx_lb,y_train,b_idxs,rd,h):
    
    pca = PCA(n_components=2)
    transformed = pca.fit_transform(X=X_train)
    x_component = transformed[:, 0]
    
    plt.figure(figsize=[9,6])
    plt.scatter(transformed[:, 0][~idx_lb],transformed[:, 1][~idx_lb],label="unlabelled points",c="brown")
    plt.scatter(transformed[:, 0][idx_lb],transformed[:, 1][idx_lb],label="labelled points")
    plt.scatter(transformed[:, 0][b_idxs],transformed[:, 1][b_idxs],label="new points added",c="yellow")
    plt.legend()
    plt.title(f"points selected after {rd} rounds")

In [3]:
boston = datasets.load_boston()
df = pd.DataFrame(boston.data, columns = boston.feature_names)
df["target"]=boston.target
y_boston=df.target
X_boston=df.drop("target",axis=1)
#scaler = RobustScaler()
#X_boston=pd.DataFrame(scaler.fit_transform(X_boston)).set_axis(X_boston.columns, axis=1)
y_boston=torch.Tensor(y_boston).view(len(y_boston),1).float()
X_boston=torch.Tensor(X_boston.values).float()


columns_names=["Frequency","Angle of attack","Chord length","Free-stream velocity","Suction side displacement thickness","sound pressure level"]
airfoil=pd.read_csv('C:/Users/matth/jupyter codes/Recherche/datasets/airfoil_self_noise.dat',sep='\t',names=columns_names)
y_airfoil=airfoil["sound pressure level"]
X_airfoil=airfoil.drop("sound pressure level",axis=1)
y_airfoil=torch.Tensor(y_airfoil).view(len(y_airfoil),1).float()
X_airfoil=torch.Tensor(X_airfoil.values).float()


energy=pd.read_csv('C:/Users/matth/jupyter codes/Recherche/datasets/energy efficiency.csv')
y_energy=energy["Y2"]
X_energy=energy.drop(["Y2","Y1"],axis=1)
y_energy=torch.Tensor(y_energy).view(len(y_energy),1).float()
X_energy=torch.Tensor(X_energy.values).float()


maintenance=pd.read_csv('C:/Users/matth/jupyter codes/Recherche/datasets/conditions_based_maintenance/data.txt',sep="   ", header=None)
y_maintenance=maintenance[17]
X_maintenance=maintenance.drop([16,17],axis=1)
y_maintenance=torch.Tensor(y_maintenance).view(len(y_maintenance),1).float()
X_maintenance=torch.Tensor(X_maintenance.values).float()


proportion=0.2
scaler = MinMaxScaler()
X=X_airfoil
y=y_airfoil

X=torch.Tensor(scaler.fit_transform(X))


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=proportion)

#generating testset

#For Boston
#scaler_test = RobustScaler()
#y_train=torch.Tensor(scaler_test.fit_transform(y_train))

#taking limited number of datas to make the query process run faster
#X_train=X_train[:100]
#y_train=y_train[:100]

X_train.shape

C:\Users\matth\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (

torch.Size([1202, 5])

In [4]:
def pi(mu, std, y_max, beta):
    Z = (mu - y_max - beta) / std
    return norm.cdf(Z)

In [8]:
nb_initial_labelled_datas=int(0.02*X_train.shape[0])
n_query_each_round=int(0.02*X_train.shape[0])#int(0.02*X_train.shape[0])
num_round=int(np.ceil((len(y_train)-nb_initial_labelled_datas)/n_query_each_round))
dim_input=13
total_epoch_h=100
batch_size_train=len(X_train)
lr_h=0.001
beta=10

liste_results=[]
for m in range(5):

    n_pool = len(y_train)
    n_test = len(y_test)
    idxs_lb = np.zeros(n_pool, dtype=bool)
    idxs_tmp = np.arange(n_pool)
    np.random.shuffle(idxs_tmp)
    idxs_lb[idxs_tmp[:nb_initial_labelled_datas]] = True


    config_NN=[2,16,32]

    RMSE_list=[]
    MAPE_list=[]
    MAE_list=[]


        # Gaussian process regression
    kernel = RBF(length_scale=0.1, length_scale_bounds=(1e-8, 1e8)) \
             + WhiteKernel()

    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=5)

    for rd in range(1,num_round+1):

        print('\n================Round {:d}==============='.format(rd))

        if rd==num_round:
                last_query=True

        idx_lb_train = np.arange(n_pool)[idxs_lb]
        idx_ulb_train = np.arange(n_pool)[~idxs_lb]
        print(len(X_train[idx_lb_train]),len(X_train[idx_ulb_train]))
        #print(X_train[idx_lb_train])
        #print(len(trainloader_labelled))

        gpr.fit(X_train[idx_lb_train],y_train[idx_lb_train])

        if rd==num_round:
            print("hey")
            idxs_lb[:]= True
        else:

            #print(std)
            #print(np.sort(std))
            y_pred, y_std = gpr.predict(X_train[idx_ulb_train], return_std=True)
            y_max = np.max(y_train[idx_lb_train])
            pi_values = pi(y_pred, y_std, y_max, beta)
                                        
            query_idx= np.argsort(pi_values)[-n_query_each_round:]
            #print(query_idx)
            print(idx_ulb_train[query_idx])
            idxs_lb[idx_ulb_train[query_idx]] = True
            display_chosen_labelled_datas_PCA(X_train,idxs_lb,y_train,idx_ulb_train[query_idx],rd,h)

        y_pred=h(X_test)
        RMSE_list.append(np.sqrt(mean_squared_error(y_pred.detach().numpy(),y_test)))
        MAPE_list.append(mean_absolute_percentage_error(y_pred.detach().numpy(),y_test))
        MAE_list.append(mean_absolute_error(y_pred.detach().numpy(),y_test))

    print('\n================Final training===============')

    idx_lb_train = np.arange(n_pool)[idxs_lb]
    idx_ulb_train = np.arange(n_pool)[~idxs_lb]
    trainset_labelled=myData(X_train[idx_lb_train],y_train[idx_lb_train])
    trainloader_labelled= DataLoader(trainset_labelled,shuffle=True,batch_size=batch_size_train)

    for epoch in range(total_epoch_h):

            for i,data in enumerate(trainloader_labelled,0):
                label_x, label_y=data
                opti_h.zero_grad() 
                lb_out = h(label_x)
                #print(label_x)
                h_descent=torch.mean((lb_out-label_y)**2)
                h_descent.backward()
                opti_h.step()

    y_pred=h(X_test)
    RMSE_list.append(np.sqrt(mean_squared_error(y_pred.detach().numpy(),y_test)))
    MAPE_list.append(mean_absolute_percentage_error(y_pred.detach().numpy(),y_test))
    MAE_list.append(mean_absolute_error(y_pred.detach().numpy(),y_test))
    liste_results.append([MAE_list,MAPE_list,RMSE_list])


================Round 1===============
24 1178


C:\Users\matth\AppData\Local\Temp\ipykernel_173096\1425837751.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  reg.fit(X_train[idx_lb_train],y_train[idx_lb_train])


TypeError: predict() got an unexpected keyword argument 'return_std'